# Pruning SSD Model
This notebook shows how to reduce the size of a model by pruning its parameters. It assumes 
that a trained model already exists in the ```Models``` directory. Please refer to the notebook [Object Detection with SSD](SSD.ipynb) for more info about using a pretrained SSD model.

If you want to prune a Low-Rank model, you can use [this](SSD-Reduce.ipynb) notebook
to reduce the number of parameters in ```SSD```.

## Load and evaluate the original pretrained model

In [1]:
from fireball import Model, myPrint
from fireball.datasets.coco import CocoDSet
gpus = "0,1,2,3"

myPrint('\nPreparing Coco dataset ... ', False)
trainDs,testDs = CocoDSet.makeDatasets('Train,Test', batchSize=128, resolution=512, keepAr=False, numWorkers=4)
trainDs.batchSize = 64
myPrint('Done.')

Model.config.maxDetectionsPerImage = 200
Model.config.maxDetectionPerClass = 100
Model.config.scoreThreshold = 0.01

# orgFileName = "Models/SSD512.fbm"    # Original model
# orgFileName = "Models/SSD512R.fbm"   # Reduced
orgFileName = "Models/SSD512RR.fbm"    # Reduced - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.initSession()
results = model.evaluate()


Preparing Coco dataset ... Done.

Reading from "Models/SSD512RR.fbm" ... Done.
Creating the fireball model "SSD512" ... Done.
  Processed 5000 Sample. (Time: 93.72 Sec.)                              

Evaluating inference results for 5000 images ... 
  Calculating IoUs - Done (8.1 Seconds)                       
  Finding matches - Done (116.0 Seconds)                     
  Processing the matches - Done (3.8 Seconds)                    
Done (127.9 Seconds)

Average Precision (AP):
    IoU=0.50:0.95   Area: All      MaxDet: 100  = 0.248
    IoU=0.50        Area: All      MaxDet: 100  = 0.465
    IoU=0.75        Area: All      MaxDet: 100  = 0.241
    IoU=0.50:0.95   Area: Small    MaxDet: 100  = 0.096
    IoU=0.50:0.95   Area: Medium   MaxDet: 100  = 0.292
    IoU=0.50:0.95   Area: Large    MaxDet: 100  = 0.365
Average Recall (AR):
    IoU=0.50:0.95   Area: All      MaxDet: 1    = 0.230
    IoU=0.50:0.95   Area: All      MaxDet: 10   = 0.351
    IoU=0.50:0.95   Area: All      MaxDet:

## Pruning the model
Here we prune the model using the ``pruneModel`` class method of the model.

In [7]:
prunedFileName = orgFileName.replace('.fbm', 'P.fbm')  # Append 'P' to the filename for "Pruned"
pResults = Model.pruneModel(orgFileName, prunedFileName, mseUb=.000005)


Reading model parameters from "Models/SSD512RR.fbm" ... Done.
Pruning 92 tensors (mseUb=0.000005)
    Tensor 1 of 92 Shape: 3x3x3x64 .......... Done. 112 Pruned < 0.016198, MSE=0.0000049, Reduced: 3.3%)
    Tensor 2 of 92 Shape: 64 ................ Ignored. (1-D Tensor)
    Tensor 3 of 92 Shape: 3x3x64x64 ......... Done. 8605 Pruned < 0.008191, MSE=0.0000050, Reduced: 20.2%)
    Tensor 4 of 92 Shape: 64 ................ Ignored. (1-D Tensor)
    Tensor 5 of 92 Shape: 3x3x64x128 ........ Done. 21057 Pruned < 0.007482, MSE=0.0000050, Reduced: 25.4%)
    Tensor 6 of 92 Shape: 128 ............... Ignored. (1-D Tensor)
    Tensor 7 of 92 Shape: 3x3x128x128 ....... Done. 46643 Pruned < 0.007123, MSE=0.0000050, Reduced: 28.5%)
    Tensor 8 of 92 Shape: 128 ............... Ignored. (1-D Tensor)
    Tensor 9 of 92 Shape: 3x3x128x136 ....... Done. 40012 Pruned < 0.007717, MSE=0.0000050, Reduced: 22.4%)
    Tensor 10 of 92 Shape: 1x1x136x256 ...... Done. 5733 Pruned < 0.009664, MSE=0.0000050, Re

    Tensor 90 of 92 Shape: 324 .............. Ignored. (1-D Tensor)
    Tensor 91 of 92 Shape: 3x3x256x16 ....... Done. 6779 Pruned < 0.009020, MSE=0.0000050, Reduced: 15.3%)
    Tensor 92 of 92 Shape: 16 ............... Ignored. (1-D Tensor)
Pruning process complete (2.21 Sec.)
Now saving to "Models/SSD512RRP.fbm" ... Done.

Number of parameters: 23,147,860 -> 15,799,149 (7,348,711 pruned)
Model File Size: 92,598,147 -> 66,097,079 bytes


## Evaluate the pruned model
Compare the new number of parameters with the original. Let's see the impact of this reduction to the performance of the model.

In [8]:
model = Model.makeFromFile(prunedFileName, testDs=testDs, gpus=gpus)   
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/SSD512RRP.fbm" ... Done.
Creating the fireball model "SSD512" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 512x512x3    512 512 3     None                      0          
S1_L1_CONV       512 512 3     KSP: 3 1 s               512 512 64    ReLU                      1,680      
S1_L2_CONV       512 512 64    KSP: 3 1 s               256 256 64    ReLU     MP(KSP):2 2 s    28,323     
S2_L1_CONV       256 256 64    KSP: 3 1 s               256 256 128   ReLU                      52,799     
S2_L2_CONV       256 256 128   KSP: 3 1 s               128 128 128   ReLU     MP(KSP):2 2 s    100,941    
S3_L1_CONV       128 128 128   KSP: 3 1 s, LR136        128 128 256   ReLU                      145,999    
S3_L2_CONV       128 128 256   KSP: 3 1 s

## Re-training after pruning
Here we make a new model for re-training from the file created above. We then call the ```train``` method of the model to start the re-training.

After re-training, we run the ```evaluate``` function again to see how the re-training improved the performance
of the model.

The re-trained model is then saved to a file appending an 'R' letter (for Re-trained) to the end of the pruned model file name.

In [9]:

model = Model.makeFromFile(prunedFileName, trainDs=trainDs, testDs=testDs,
                           numEpochs=5,
                           learningRate=(0.002, 0.0004),
                           optimizer="Momentum",
                           gpus=gpus)
model.initSession()
model.printLayersInfo()
model.printNetConfig()
model.train()
results = model.evaluate(topK=5)

retrainedFileName = prunedFileName.replace('.fbm', 'R.fbm')  # Append 'R' to the filename for "Retrained"
model.save(retrainedFileName)


Reading from "Models/SSD512RRP.fbm" ... Done.
Creating the fireball model "SSD512" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 512x512x3    512 512 3     None                      0          
S1_L1_CONV       512 512 3     KSP: 3 1 s               512 512 64    ReLU                      1,680      
S1_L2_CONV       512 512 64    KSP: 3 1 s               256 256 64    ReLU     MP(KSP):2 2 s    28,323     
S2_L1_CONV       256 256 64    KSP: 3 1 s               256 256 128   ReLU                      52,799     
S2_L2_CONV       256 256 128   KSP: 3 1 s               128 128 128   ReLU     MP(KSP):2 2 s    100,941    
S3_L1_CONV       128 128 128   KSP: 3 1 s, LR136        128 128 256   ReLU                      145,999    
S3_L2_CONV       128 128 256   KSP: 3 1 s

## Where do I go from here?

[Quantizing SSD Model](SSD-Quantize.ipynb)

[Exporting SSD Model to ONNX](SSD-ONNX.ipynb)

[Exporting SSD Model to TensorFlow](SSD-TF.ipynb)

[Exporting SSD Model to CoreML](SSD-CoreML.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with SSD](SSD.ipynb)

[Reducing number of parameters of SSD Model](SSD-Reduce.ipynb)